### Gemini-MultiModal-usecases.ipynb File

In [ ]:
'''
Pre-requisites:
    GCP account need to be created.
    GCP project need to be linked with GCP billing/service account.
    GCP project need to be linked with GCP service account key file.(json file)
    Download json file and set env variable GOOGLE_APPLICATION_CREDENTIALS to the path of the downloaded file.
    eg: export GOOGLE_APPLICATION_CREDENTIALS="/home/user/gcp_project/data/json/service-account-file.json"
    For different models,  ref. to model garden in Vertex AI in gcp console.
'''

In [ ]:
%pip install google-cloud-aiplatform gitpython magika

In [ ]:

# Initialize Vertex AI
import vertexai
from vertexai.generative_models import GenerativeModel
vertexai.init(project="river-span-431711-k8", location="us-central1")# Load the Gemini 1.5 Pro model. (https://cloud.google.com/vertex-ai/docs/reference/python/latest/vertexai.generative_models)

# Load the Gemini 1.5 Pro model. (https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models)
multimodal_model = GenerativeModel("gemini-1.5-pro-001")
multimodal_model_flash = GenerativeModel("gemini-1.5-flash-001") # using this var later

# Generate response
contents = [ "Explain LLM" ]
response = multimodal_model.generate_content(contents)
print(response)
print(response.text)


In [ ]:
import IPython.display
display(IPython.display.Markdown(response.text))